In [48]:
from dotenv import load_dotenv
import os
import openai
import json


In [19]:
load_dotenv()
key = os.getenv("OPENAI_API_KEY")
openai.api_key = key


In [11]:
conversation_history = [ {"role": "system", "content": "You are a helpful assistant which is being used in an email application, for example: gmail, your aim is to help user only with email related tasks and some natural comverstaion if he wants or execute functions based on user input"}]

In [39]:
functions = [
    {
        "name": "send_mail",
        "description": "send email to an email",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {
                    "type": "string",
                    "description": "email address to which email needs to be sent",
                },
                "subject": {
                    "type": "string",
                    "description": "Subject of the email",
                },
                "body": {
                    "type": "string",
                    "description": "Body of the email",
                }
            },
            "required": ["to", "subject", "body"],
        },
    },
    {
        "name": "search_email",
        "description": "search all the emails of the user to find specific email that user wants",
         "parameters": {
            "type": "object",
            "properties": {
                "from_email": {
                    "type": "string",
                    "description": "from email address which can be userd to filter all emails sent by a certain email address",
                },
                "words": {
                    "type": "string",
                    "description": "text which should be there in email subject or body",
                }
            },
            "required": ["from_email", "words"],
        },
    },
    {
        "name": "change_name",
        "description": "Change the name of the user to new specified name",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "new name of the user",
                },
            },
            "required": ["name"],
        },
    },
    {
        "name": "create_folder",
        "description": "create a new folder for the user",
        "parameters": {
            "type": "object",
            "properties": {
                "folder_name": {
                    "type": "string",
                    "description": "folder name which user wants to create",
                },
            },
            "required": ["folder_name"],
        },
    },
    {
        "name": "add_rule_move_to_folder",
        "description": "add a new filter or rule which helps user to to filter his incoming mails. For example move mails coming from certain email to some folder",
         "parameters": {
            "type": "object",
            "properties": {
                "rule_name": {
                    "type": "string",
                    "description": "name of the rule user wants to create",
                },
                "from_email": {
                    "type": "string",
                    "description": "a valid emaill address for the the filter being created. Filter will be applied on all the mail coming from this email",
                },
                "folder_name": {
                    "type": "string",
                    "description": "folder name to which mails need to be moved to based on the filter",
                }
            },
            "required": ["rule_name", "from_email", "folder_name"],
         }
    }
]

In [13]:
def talk_to_gpt3(prompt, conversation_history):
    conversation_history.append({"role": "user", "content": prompt})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=conversation_history,
        functions=functions,
        function_call="auto",
    )
    answer = response.choices[0].message['content']
    conversation_history.append({"role": "assistant", "content": answer})
    return answer

In [47]:
conversation_history = [ {"role": "system", "content": "You are a helpful assistant which is being used in an email application, for example: gmail, your aim is to help user only with email related tasks and some natural comverstaion if he wants"}]

prompt = "Create a folder for me with name Important Emails"

conversation_history.append({"role": "user", "content": prompt})

In [43]:


response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=conversation_history,
    functions=functions,
    function_call="auto",
)

In [44]:
response

<OpenAIObject chat.completion id=chatcmpl-7g8ymNDFUUGMk7FMwhn6wGF8vNWec at 0x1069e75b0> JSON: {
  "id": "chatcmpl-7g8ymNDFUUGMk7FMwhn6wGF8vNWec",
  "object": "chat.completion",
  "created": 1690278648,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "create_folder",
          "arguments": "{\n\"folder_name\": \"Important Emails\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 380,
    "completion_tokens": 16,
    "total_tokens": 396
  }
}

In [45]:
message = response.choices[0].message

In [ ]:
if message['content']:
    answer = message['content']
    conversation_history.append({"role": "assistant", "content": answer})
elif message['function_call']:
    function_name = message["function_call"]["name"]
    function_args = json.loads(message["function_call"]["arguments"])
    result = "Task has been done"
    conversation_history.append({"role": "function", "name":function_name ,"content": result })